# 08. Produce alt contigs for omnidopsis project

specifically for split orthogroups which do not contain a sequence from at9852

In [1]:
conda activate dl20

In [2]:
dl20=(
at6137 at6923 at6929 at7143
at8285 at9104 at9336 at9503
at9578 at9744 at9762 at9806
at9830 at9847 at9852 at9879
at9883 at9900
)

In [3]:
mkdir -p tmp/altcontigs output/03_alt-contigs/

## Get list of orthogroups *not* in 9852

In [4]:
# list of split orthogroups that don't contain at9852
grep "at9852" -L ../luisa-misc/panrg-for-luisas-talk/input/all_fasta_70/* \
 > tmp/altcontigs/split_orthologs_missing_at9852.txt
 
grep "at9744" -L ../luisa-misc/panrg-for-luisas-talk/input/all_fasta_70/* \
 > tmp/altcontigs/split_orthologs_missing_at9852.txt

grep "at7143" -L ../luisa-misc/panrg-for-luisas-talk/input/all_fasta_70/* \
 > tmp/altcontigs/split_orthologs_missing_at9852.txt

Cat this sequences to get a list of sequences from which to select alt contigs

## Prepare TE annotation

In [5]:
# Pansn rename the reduced TE gffs
# TODO: move this to the original pansn-naming notebook

THIS ONE IS NOT NEEDED FOR NEW TE ANNOTATIONS

for accession in "${dl20[@]}"
do 
grep "^$accession" input/2022-07-16_curated-repeat-annotation-adri/pangenome_TEannotation_intacts.gff3 | \
    ./src/pansn-rename -R '_' -d '#' -D '_' -m cat \
        -r output/01_assembly/01_pansn-named/${accession}.scaffolds-v2.3.fasta.fai \
        -o output/01_assembly/01_pansn-named/${accession}_reduced_TE_anno_adri_2022-07-06.gff3 \
        -
done

tmp/altcontigs/bed/Araport_sorted_just_genes.bed## Merge TE annotation with liftoff annotation

In [6]:
for accession in "${dl20[@]}"
do 
    # Extract just the genes from the liftoff annotations, convert to bed
    awk '$3 == "gene" && $9 ~ /.*Note=protein_coding_gene.*/{print $0}' \
        <output/01_assembly/01_pansn-named/${accession}.liftoff-v2.3.gff \
    | sed -e 's/;.*//' \
    | gff2bed > tmp/altcontigs/${accession}_liftoff-genes.bed
    
    # Merge the TE annotations with the liftoff such that you can retrieve the fasta sequence for 
    # all the gene models of interest with a nominal amount of flanking sequence that does not read
    # into the neighboring genes or TEs. Then convert to bed
    bedtools subtract -a ../TE_annotation/output/New_TE_annotations_from_Adrian/split_by_accession/${accession}_TEintact.gff3 \
        -b output/02_annotation/01_original-annotation-merged/${accession}.liftoff-v2.3.gff3 \
        -A \
    | gff2bed \
      > tmp/altcontigs/${accession}_reduced-te_sans_ingene_te.bed
   
    cat tmp/altcontigs/${accession}_liftoff-genes.bed \
        tmp/altcontigs/${accession}_reduced-te_sans_ingene_te.bed \
    | bedtools sort \
        -i - \
        > tmp/altcontigs/${accession}_te_and_liftoff_annotations_sorted.bed
    
done

## Extract genes + flanking sequences

In [7]:
for accession in "${dl20[@]}"
do
    python3 src/flanking_regions_bed_editor.py \
        -g tmp/altcontigs/${accession}_te_and_liftoff_annotations_sorted.bed \
        -f output/01_assembly/01_pansn-named/${accession}.scaffolds-v2.3.fasta.fai \
        -m 300 \
        > tmp/altcontigs/${accession}_te_and_liftoff_annotations_withflanking.bed

    bedtools getfasta -s -name \
       -fi output/01_assembly/01_pansn-named/${accession}.scaffolds-v2.3.fasta \
       -bed tmp/altcontigs/${accession}_te_and_liftoff_annotations_withflanking.bed \
       > tmp/altcontigs/${accession}_all-annotated-features-with-flanking.fasta
done

## Find which araport IDs match which augustus IDs in the orthofinder

In [8]:
for accession in "${dl20[@]}"
do
    awk '$3 == "gene"{print $0}' \
        <output/01_assembly/01_pansn-named/${accession}.augustus-v2.3.gff3 \
    | gff2bed > tmp/altcontigs/${accession}.augustus-v2.3.bed
        
    bedtools intersect -wo \
       -a tmp/altcontigs/${accession}_liftoff-genes.bed \
       -b tmp/altcontigs/${accession}.augustus-v2.3.bed \
    | cut -f 4,14 \
       > tmp/altcontigs/${accession}_intersect_augustus_and_liftoff.txt
    
done

In [8]:
ls tmp/altcontigs/*_all-annotated-features-with-flanking.fasta >tmp/altcontigs/flankingfasta.fofn
ls tmp/altcontigs/*_intersect_augustus_and_liftoff.txt >tmp/altcontigs/intersection.fofn

In [9]:
python3 src/pick_alt_contigs.py \
    -o output/03_alt-contigs/alt_contigs_20230110.fasta \
    -f tmp/altcontigs/split_orthologs_missing_at9852.txt \
    -F tmp/altcontigs/flankingfasta.fofn \
    -b tmp/altcontigs/intersection.fofn

OG0020285 at6137_g5522
OG0007265 at6137_g6115
OG0022220 at6137_g21652
OG0020405 at6929_g6217
OG0022364 at6929_g22410
OG0026014 at6929_g26254
OG0028700 at6929_g29227
OG0024998 at6929_g31604
OG0026847 at8285_g11465
OG0000620 at9104_g5957
OG0020448 at9104_g6550
OG0027951 at9104_g6873
OG0027952 at9104_g6874
OG0020726 at9104_g9493
OG0028295 at9104_g28785
OG0000324 at9104_g29306
OG0027372 at9336_g5529
OG0027136 at9336_g29934
OG0002663 at9336_g30113
OG0027953 at9503_g6879
OG0024379 at9503_g29586
OG0001611 at9578_g5520
OG0026328 at9578_g29845
OG0001142 at9744_g5555
OG0000695 at9744_g30441
OG0027994 at9744_g30442
OG0028258 at9762_g5710
OG0024861 at9762_g6924
OG0002657 at9762_g29124
OG0002449 at9806_g5466
OG0007341 at9806_g6211
OG0024899 at9806_g13595
OG0021707 at9830_g25928
OG0017972 at9830_g29540
OG0000122 at9847_g17563
OG0002638 at9847_g26643
OG0002178 at9847_g29787
OG0029188 at9847_g29789
OG0022223 at9852_g21399
OG0016891 at9852_g26174
OG0017991 at9852_g29361
OG0026553 at9879_g2661
OG0000197

In [11]:
wc -l output/03_alt-contigs/alt_contigs_20230110.fasta

98 output/03_alt-contigs/alt_contigs_20230110.fasta


## Make the bedfiles for the reference genomes

In [13]:
references=(
at7143 at9852 at9744
)

In [14]:
mkdir -p tmp/altcontigs/bed

In [21]:
gff2bed < output/01_assembly/01_pansn-named/${references}.augustus-v2.3.gff3 > tmp/altcontigs/bed/${references}_sorted.bed
grep "gene" tmp/altcontigs/bed/${references}_sorted.bed > tmp/altcontigs/bed/${references}_sorted_just_genes.bed

In [31]:
gff2bed < output/01_assembly/01_pansn-named/Araport.gff3 > tmp/altcontigs/bed/Araport_sorted.bed
grep "Araport11"$'\t'"gene" tmp/altcontigs/bed/Araport_sorted.bed > tmp/altcontigs/bed/Araport_sorted_just_genes.bed